In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from FeatureSelection import *
from utils import *
%matplotlib inline

In [10]:
# Retrieve Data
data = retrieve_data()
train = data['train']
test = data['test']
train_num = data['train_num']
y_feature = 'SalePrice'

# Imputating data with KNN: Try doing this with make pipleline
for feature in data['train_num_missing']:
    train_num[feature] = fillNaWithKNN(feature, train_num, y_feature)
print(train_num.isna().any().values)
train_num.to_csv('imputed-train.csv', index=False)

all_features_list = train.corr()['SalePrice'][1:-1].keys().to_list()

poly_train = polynomial_options(train, all_features_list)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False]


In [ ]:
poly_train

In [6]:
l = poly_train.corr()['SalePrice'].nlargest(20)[1:].keys().to_list()

In [3]:
handPicked_features = [
    'OverallQual^3', 'OverallQual^4', 'OverallQual^2', 'OverallQual', 
    'GrLivArea'
]
doubted_features = poly_train.corr()['SalePrice'].nlargest(50)[20:26].keys().to_list()

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [35]:
"""
    Implementing mini-batch gradient descent:
        - Having a number for batch-size
        - Going through the whole dataset in each batch
        - passing the parameters along from each batch to the other one
        set_params(**args) /get_params
"""
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(eta0=0.35, max_iter=1600, n_iter_no_change=45, random_state=31)
X_train, X_test, y_train, y_test = train_test_split(poly_train[l], train[y_feature], test_size=0.33, random_state=42)

# sgd.fit(poly_train[l], train['SalePrice'])
sgd.fit(X_train, y_train)
print(r2_score(y_true=y_test, y_pred=sgd.predict(X_test)))
y_pred = sgd.predict(X_test)
# print(sgd.score(y_test, y_pred))
sgd.fit(poly_train[l], train[y_feature])

-1.0316868836167893e+49


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.35, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1600,
             n_iter_no_change=45, penalty='l2', power_t=0.25, random_state=31,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [112]:
from sklearn.preprocessing import normalize
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

test_num = data['test_num']
X_test = test_num[l]
# Imputating the test data
for feat in l:
    for n in range(0, X_test['GarageCars'].size):
        types = ['int64', 'float64']
        Type = X_test['GarageCars'].iloc[n].dtype
        if not(Type in types):
            X_test['GarageCars'].iloc[n] = X_test['GarageCars'].mean()
X_test = X_test.fillna(X_test.mean())

X_test = polynomial_options(test_num, test.corr()[1:-1])
X_test = (X_test - X_test.mean()) / X_test.mean()
X_test = X_test[l]

In [120]:
for n in range(0, X_test['GarageCars'].size):
    types = ['int64', 'float64']
    Type = X_test['GarageCars'].iloc[n].dtype
    if not(Type in types):
        X_test['GarageCars'].iloc[n] = X_test['GarageCars'].mean()
    if n == 1116: print('s', X_test['GarageCars'].iloc[n].dtype, X_test['GarageCars'].iloc[n] == np.NaN)
X_test = X_test.fillna(X_test.mean())

s float64 False


In [113]:
X_test.isna().any()

OverallQual^3     False
OverallQual^2     False
OverallQual       False
GrLivArea         False
GarageCars^2       True
GrLivArea^2       False
GarageCars^3       True
GarageCars         True
GarageArea         True
TotalBsmtSF        True
GarageArea^2       True
1stFlrSF          False
FullBath^2        False
FullBath^3        False
FullBath          False
1stFlrSF^2        False
TotRmsAbvGrd      False
TotRmsAbvGrd^2    False
YearBuilt^3       False
dtype: bool

In [101]:
pred = sgd.predict(X_test)

output = pd.DataFrame({'Id': test.Id,
                      'SalePrice': pred})
output.to_csv('submission.csv', index=False)

In [102]:
bench = load_bench_data()
bench

,SalePrice
Id,
1,122430.677167
2,164746.246000
3,176869.848667
4,203227.895500
5,171174.603333
...,...
1455,83863.886667
1456,89974.796667
1457,170363.683500


In [104]:
r2_score(bench['SalePrice'], pred)

-7.996790150161541e+27